<a href="https://colab.research.google.com/github/k-washi/3d-Bonet-Boost/blob/master/BoNet_S3DIS_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D - BoNetのスタート

データセットは、[Stanford 2D-3D-Semantics Dataset](http://buildingparser.stanford.edu/dataset.html)

h5データへの変更は、[pqhieu/jsis3d](https://github.com/pqhieu/jsis3d/blob/master/scripts/prepare_h5.py)にて提供されている。
また、データセットのh5 versionは、以下の3D-BoNetのリポジトリからダウンロードできる。


# 3D-BoNetのインストール
[Learning Object Bounding Boxes for 3D Instance Segmentation on Point Clouds](https://arxiv.org/abs/1906.01140)

https://github.com/Yang7879/3D-BoNet
---



In [0]:
!rm -rf 3D-BoNet

In [5]:
!git clone https://github.com/Yang7879/3D-BoNet.git

Cloning into '3D-BoNet'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 198 (delta 22), reused 2 (delta 0), pack-reused 151
Receiving objects: 100% (198/198), 71.74 MiB | 3.31 MiB/s, done.
Resolving deltas: 100% (84/84), done.


# 補助パッケージの取得

https://github.com/k-washi/3d-Bonet-Boost

In [0]:
!rm -rf 3d-Bonet-Boost

In [60]:
!git clone https://github.com/k-washi/3d-Bonet-Boost.git

Cloning into '3d-Bonet-Boost'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 33 (delta 16), reused 21 (delta 8), pack-reused 0
Unpacking objects: 100% (33/33), done.


#python ライブラリの確認

tensorflow 1.4

In [37]:
%tensorflow_version 1.x
!pip show tensorflow

Name: tensorflow
Version: 1.4.0
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, wheel, protobuf, six, tensorflow-tensorboard, enum34
Required-by: stable-baselines, magenta, fancyimpute


もし、versionが1.4出ない場合は、変更
もし、RESTAER RUNTIMEが出たら再起動

In [0]:
!pip install tensorflow==1.4 > NILL

# データセットのダウンロード

Google Driveにデータセットを置くと仮定

/content/drive/My Drive/data/Data_S3DIS.zip

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/': No such file or directory


データを解凍

In [9]:
!apt-get -y install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
!mkdir s3dis
!unzip /content/drive/My\ Drive/data/Data_S3DIS.zip -d s3dis > NILL

# 3D-BoNet: Compile tensorflow operation by cuda 

In [0]:
!cp 3d-Bonet-Boost/cu_compile.py 3D-BoNet/tf_ops/

コンパイルに必要なコマンドを出力

In [26]:
!nvcc --version
!which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
/usr/local/cuda/bin/nvcc


compileに必要なcudaディレクトリとGC上で実行することを引数で与える

In [63]:
!python 3D-BoNet/tf_ops/cu_compile.py --cuda /usr/local/cuda --googleColab

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or 

上のpythonの出力コマンドをコピーして実行 (2020/3/9時点では、以下のコマンド)

In [64]:
!/usr/local/cuda/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!/usr/local/cuda/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_so.so -shared -fPIC -I /usr/local/lib/python3.6/dist-packages/tensorflow/include -I /usr/local/cuda/include -I /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/nsync/public -L /usr/local/lib/python3.6/dist-packages/tensorflow -lcudart -L /usr/local/cuda/lib64/ -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling.cpp 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_so.so -shared -fPIC -I /usr/local/lib/python3.6/dist-packages/tensorflow/include -I /usr/local/cuda/include -I /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/nsync/public -L /usr/local/lib/python3.6/dist-packages/tensorflow -lcudart -L /usr/local/cuda/lib64/ -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate.cpp -o 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate_so.so -shared -fPIC -I /usr/local/lib/python3.6/dist-packages/tensorflow/include -I /usr/local/cuda/include -I /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/nsync/public -L /usr/local/lib/python3.6/dist-packages/tensorflow -lcudart -L /usr/local/cuda/lib64/ -O2

3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp: In lambda function:
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:22:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
         c->WithRank(c->input(1), 3, &dims2);
                                            ^
In file included from 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:8:0:
/usr/local/lib/python3.6/dist-packages/tensorflow/include/tensorflow/core/framework/shape_inference.h:377:10: note: declared here
   Status WithRank(ShapeHandle shape, int64 rank,
          ^~~~~~~~
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp: In lambda function:
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:47:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceCo